# Introduction: DDOT tutorial

* __What is an ontology?__ An ontology is a hierarchical arrangement of two types of nodes: (1)
genes at the leaves of the hierarchy and (2) terms at intermediate
levels of the hierarchy. The hierarchy can be thought of as directed
acyclic graph (DAG), in which each node can have multiple children or
multiple parent nodes. DAGs are a generalization of trees
(a.k.a. dendogram), where each node has at most one parent.

* __What is DDOT?__ The DDOT Python package provides many functions for assembling,
analyzing, and visualizing ontologies.  The main functionalities are
implemented in an object-oriented manner by an "Ontology" class, which handles ontologies that are data-driven as well as those
that are manually curated like the Gene Ontology.

* __What to do after reading this tutorial__ Check out a complete list of functions in the [Ontology class](http://ddot.readthedocs.io/en/latest/ontology.html) and a list of [utility functions](http://ddot.readthedocs.io/en/latest/utils.html) that may help you build more concise pipelines. Also check out [example Jupyter notebooks](https://github.com/michaelkyu/ddot/tree/master/examples) that contain pipelines for downloading and processing the Gene Ontology and for inferring data-driven gene ontologies of diseases

In [ ]:
# Import Ontology class from DDOT package
import ddot
from ddot import Ontology

# Creating an Ontology object

* An object of the Ontology class can be created in several ways.
* In this tutorial, we will construct and analyze the toy ontology shown below.

<img src="https://raw.githubusercontent.com/michaelkyu/ddot/master/docs/toy_ontology.png" height="250" align="left">

### Create ontology through the \_\_init\_\_ constructor

In [ ]:
# Connections from child terms to parent terms
hierarchy = [('S3', 'S1'),
             ('S4', 'S1'),
             ('S5', 'S1'),
             ('S5', 'S2'),
             ('S6', 'S2'),
             ('S1', 'S0'),
             ('S2', 'S0')]

# Connections from genes to terms
mapping = [('A', 'S3'),
           ('B', 'S3'),
           ('C', 'S3'),
           ('C', 'S4'),
           ('D', 'S4'),
           ('E', 'S5'),
           ('F', 'S5'),
           ('G', 'S6'),
           ('H', 'S6')]

# Construct ontology
ont = Ontology(hierarchy, mapping)

### Create an ontology from a tab-delimited table or Pandas dataframe

In [ ]:
ont.to_table('toy_ontology.txt')

In [ ]:
ont = Ontology.from_table('toy_ontology.txt')

### From the Network Data Exchange (NDEx).
* It is strongly recommended that you create a free account on NDEx in order to keep track of your own ontologies.
* Note that there are two NDEx servers: the main one at http://ndexbio.org/ and a test server for prototyping your code at http://test.ndexbio.org. Each server requires a separate user account. While you get familiar with DDOT, we recommend that you use an account on the test server.

In [2]:
# Set the NDEx server and the user account.
# This "scratch" account will work for this tutorial, but you should replace it with your own account.
ndex_server = 'http://test.ndexbio.org'
ndex_user, ndex_pass = 'scratch', 'scratch'

In [3]:
# Upload ontology to NDEx. The string after "v2/network/" is a unique identifier, which is called the UUID, of the ontology in NDEx
url, _ = ont.to_ndex(ndex_server=ndex_server, ndex_user=ndex_user, ndex_pass=ndex_pass)
print(url)

NameError: name 'ont' is not defined

In [46]:
# Download the ontology from NDEx
ont2 = Ontology.from_ndex(url)
print(ont2)


8 genes, 7 terms, 9 gene-term relations, 7 term-term relations
node_attributes: ['NodeType', 'Label', 'y_pos', 'Vis:Shape', 'Vis:Fill Color', 'name', 'Size', 'isRoot', 'x_pos', 'Vis:Size', 'Vis:Border Paint']
edge_attributes: ['EdgeType', 'Is_Tree_Edge', 'Vis:Visible', '2']


# Inspecting the structure of an ontology

An Ontology object contains seven attributes:

* ``genes`` : List of gene names
* ``terms`` : List of term names
* ``gene_2_term`` : dictionary mapping a gene name to a list of terms connected to that gene. Terms are represented as their 0-based index in ``terms``.
* ``term_2_gene`` : dictionary mapping a term name to a list or genes connected to that term. Genes are represented as their 0-based index in ``genes``.
* ``child_2_parent`` : dictionary mapping a child term to its parent terms.
* ``parent_2_child`` : dictionary mapping a parent term to its children terms.
* ``term_sizes`` : A list of each term's size, i.e. the number of unique genes contained within this term and its descendants. The order of this list is the same as ``terms``. For every ``i``, it holds that ``term_sizes[i] = len(self.term_2_gene[self.terms[i]])``

In [47]:
ont.genes

['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H']

In [48]:
ont.terms

['S0', 'S1', 'S2', 'S3', 'S4', 'S5', 'S6']

In [49]:
ont.gene_2_term

{'A': [3],
 'B': [3],
 'C': [3, 4],
 'D': [4],
 'E': [5],
 'F': [5],
 'G': [6],
 'H': [6]}

In [50]:
ont.term_2_gene

{'S3': [0, 1, 2],
 'S4': [2, 3],
 'S5': [4, 5],
 'S6': [6, 7],
 'S0': [],
 'S1': [],
 'S2': []}

In [51]:
ont.child_2_parent

{'S1': ['S0'],
 'S2': ['S0'],
 'S3': ['S1'],
 'S4': ['S1'],
 'S5': ['S1', 'S2'],
 'S6': ['S2'],
 'S0': []}

In [54]:
ont.parent_2_child

{'S0': ['S1', 'S2'],
 'S1': ['S3', 'S4', 'S5'],
 'S2': ['S5', 'S6'],
 'S3': [],
 'S4': [],
 'S5': [],
 'S6': []}

Alternatively, the hierarchical connections can be viewed as a binary matrix, using `Ontology.connected()`

In [55]:
conn = ont.connected()
np.array(conn, dtype=np.int32)

array([[1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1]], dtype=int32)

A summary of an Ontology’s object, i.e. the number of genes, terms, and connections, can be printed `print(ont)`

In [56]:
print(ont)

8 genes, 7 terms, 9 gene-term relations, 7 term-term relations
node_attributes: []
edge_attributes: []


# Manipulating the structure of an ontology

DDOT provides several convenience functions for processing Ontologies into a desirable structure. Currently, there are no functions for adding genes and terms. If this is needed, then we recommend creating a new Ontology or manipulating the contents in a different library, such as NetworkX or igraph, and transforming the results into Ontology.

In [60]:
# Renaming genes and terms.
ont2 = ont.rename(genes={'A' : 'A_alias'}, terms={'S0':'S0_alias'})
ont2.to_table()

,Parent,Child,EdgeType
0,S0_alias,S1,Child-Parent
1,S0_alias,S2,Child-Parent
2,S1,S3,Child-Parent
3,S1,S4,Child-Parent
4,S1,S5,Child-Parent
5,S2,S5,Child-Parent
6,S2,S6,Child-Parent
7,S3,A_alias,Gene-Term
8,S3,B,Gene-Term
9,S3,C,Gene-Term


### Delete S1 and G while preserving transitive connections

<img src="https://raw.githubusercontent.com/michaelkyu/ddot/master/docs/toy_ontology_delete_transitive.png" height="250" align="left">

In [61]:
ont2 = ont.delete(to_delete=['S1', 'G'])
print(ont2)

7 genes, 6 terms, 8 gene-term relations, 6 term-term relations
node_attributes: []
edge_attributes: []


### Delete S1 and G (don't preserve transitive connections)

<img src="https://raw.githubusercontent.com/michaelkyu/ddot/master/docs/toy_ontology_delete_not_transitive.png" height="250" align="left">

In [62]:
ont2 = ont.delete(to_delete=['S1', 'G'], preserve_transitivity=False)
print(ont2)

7 genes, 6 terms, 8 gene-term relations, 3 term-term relations
node_attributes: []
edge_attributes: []


### Propagate gene-term connections

<img src="https://raw.githubusercontent.com/michaelkyu/ddot/master/docs/toy_ontology_propagate_gene_term.png" height="250" align="left">

In [63]:
ont2 = ont.propagate(direction='forward', gene_term=True, term_term=False)
print(ont2)

# Remove all transitive connections, and maintain only a parsimonious set of connections
ont3 = ont2.propagate(direction='reverse', gene_term=True, term_term=False)

8 genes, 7 terms, 27 gene-term relations, 7 term-term relations
node_attributes: []
edge_attributes: []


### Propagate term-term connections

<img src="https://raw.githubusercontent.com/michaelkyu/ddot/master/docs/toy_ontology_propagate_term_term.png" height="250" align="left">

In [64]:
ont2 = ont.propagate(direction='forward', gene_term=False, term_term=True)
print(ont2)

# Remove all transitive connections, and maintain only a parsimonious set of connections
ont3 = ont2.propagate(direction='reverse', gene_term=False, term_term=True)

8 genes, 7 terms, 9 gene-term relations, 11 term-term relations
node_attributes: []
edge_attributes: []


### Take the subbranch consisting of all term and genes under S1

<img src="https://raw.githubusercontent.com/michaelkyu/ddot/master/docs/toy_ontology_S1.png" height="250" align="left">

In [65]:
ont2 = ont.focus(branches=['S1'])
print(ont2)

Genes and Terms to keep: 10
6 genes, 4 terms, 7 gene-term relations, 3 term-term relations
node_attributes: []
edge_attributes: []


# Inferring a data-driven ontology

An ontology can also be inferred in a data-driven manner based on an input set of node-node similarities.

In [4]:
sim, genes = ont.flatten()
print(genes)
print(sim)

['A' 'B' 'C' 'D' 'E' 'F' 'G' 'H']
[[ 1.41503751  1.41503751  1.41503751  0.41503751  0.41503751  0.41503751 -0.         -0.        ]
 [ 1.41503751  1.41503751  1.41503751  0.41503751  0.41503751  0.41503751 -0.         -0.        ]
 [ 1.41503751  1.41503751  2.          2.          0.41503751  0.41503751 -0.         -0.        ]
 [ 0.41503751  0.41503751  2.          2.          0.41503751  0.41503751 -0.         -0.        ]
 [ 0.41503751  0.41503751  0.41503751  0.41503751  2.          2.          1.          1.        ]
 [ 0.41503751  0.41503751  0.41503751  0.41503751  2.          2.          1.          1.        ]
 [-0.         -0.         -0.         -0.          1.          1.          2.          2.        ]
 [-0.         -0.         -0.         -0.          1.          1.          2.          2.        ]]


In [5]:
ont2 = Ontology.run_clixo(sim, 0.0, 1.0, square=True, square_names=genes)

In [69]:
ont2.to_table()

,Parent,Child,EdgeType
0,S:12,S:8,Child-Parent
1,S:12,S:9,Child-Parent
2,S:13,S:9,Child-Parent
3,S:13,S:10,Child-Parent
4,S:13,S:11,Child-Parent
5,S:14,S:12,Child-Parent
6,S:14,S:13,Child-Parent
7,S:11,A,Gene-Term
8,S:11,B,Gene-Term
9,S:10,C,Gene-Term


# Ontology alignment

<img src="https://raw.githubusercontent.com/michaelkyu/ddot/master/docs/toy_ontology_alignment.png" height="250" align="left">

In [70]:
## Make a second ontology (the ontology to the right in the above diagram)

# Connections from child terms to parent terms
hierarchy = [('T3', 'T1'),
             ('T4', 'T1'),
             ('T1', 'T0'),
             ('T5', 'T0')]

# Connections from genes to terms
mapping = [('A', 'T3'),
           ('B', 'T3'),
           ('C', 'T3'),
           ('D', 'T4'),
           ('E', 'T4'),
           ('F', 'T4'),
           ('G', 'T5'),
           ('H', 'T5')]

# Construct ontology
ont_B = Ontology(hierarchy, mapping)

In [71]:
ont.align(ont_B)

collapse command: /cellar/users/mikeyu/DeepTranslate/ddot/ddot/alignOntology/collapseRedundantNodes /tmp/tmpezxw_c4d
collapse command: /cellar/users/mikeyu/DeepTranslate/ddot/ddot/alignOntology/collapseRedundantNodes /tmp/tmp65_7ptk0
Alignment command: /cellar/users/mikeyu/DeepTranslate/ddot/ddot/alignOntology/calculateFDRs /tmp/tmpuayniy01 /tmp/tmpr08_y7d3 0.05 criss_cross /tmp/tmplxrt2x6y 100 40 gene


,Term,Similarity,FDR
Term,,,
S3,T3,0.985294,0.000
S1,T1,0.913608,0.000
S6,T5,0.910000,0.040
S0,T0,0.892982,0.005
S5,T4,0.640152,0.095


# Construct ontotypes

In [72]:
# Genotypes can be represented as tuples of mutated genes
genotypes = [('A', 'B'),
             ('A', 'E'),
             ('A', 'H'),
             ('B', 'E'),
             ('B', 'H'),
             ('C', 'F'),
             ('D', 'E'),
             ('D', 'H'),
             ('E', 'H'),
             ('G', 'H')]

ontotypes = ont.get_ontotype(genotypes)
print(ontotypes)

   S0  S1  S2  S3  S4  S5  S6
0   0   0   0   2   0   0   0
1   0   0   0   1   0   1   0
2   0   0   0   1   0   0   1
3   0   0   0   1   0   1   0
4   0   0   0   1   0   0   1
5   0   0   0   1   1   1   0
6   0   0   0   0   1   1   0
7   0   0   0   0   1   0   1
8   0   0   0   0   0   1   1
9   0   0   0   0   0   0   2


In [74]:
# Genotypes can also be represented a genotype-by-gene matrix
import pandas as pd, numpy as np
genotypes_df = pd.DataFrame(np.zeros((len(genotypes), len(ont.genes)), np.float64),
                            index=['Genotype%s' % i for i in range(len(genotypes))],
                            columns=ont.genes)
for i, (g1, g2) in enumerate(genotypes):
    genotypes_df.loc['Genotype%s' % i, g1] = 1.0
    genotypes_df.loc['Genotype%s' % i, g2] = 1.0
print('Genotype matrix')
print(genotypes_df)
print()

ontotypes = ont.get_ontotype(genotypes_df, input_format='matrix')
print('Ontotype matrix')
print(ontotypes)

Genotype matrix
             A    B    C    D    E    F    G    H
Genotype0  1.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0
Genotype1  1.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0
Genotype2  1.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0
Genotype3  0.0  1.0  0.0  0.0  1.0  0.0  0.0  0.0
Genotype4  0.0  1.0  0.0  0.0  0.0  0.0  0.0  1.0
Genotype5  0.0  0.0  1.0  0.0  0.0  1.0  0.0  0.0
Genotype6  0.0  0.0  0.0  1.0  1.0  0.0  0.0  0.0
Genotype7  0.0  0.0  0.0  1.0  0.0  0.0  0.0  1.0
Genotype8  0.0  0.0  0.0  0.0  1.0  0.0  0.0  1.0
Genotype9  0.0  0.0  0.0  0.0  0.0  0.0  1.0  1.0

Ontotype matrix
            S0   S1   S2   S3   S4   S5   S6
Genotype0  0.0  0.0  0.0  2.0  0.0  0.0  0.0
Genotype1  0.0  0.0  0.0  1.0  0.0  1.0  0.0
Genotype2  0.0  0.0  0.0  1.0  0.0  0.0  1.0
Genotype3  0.0  0.0  0.0  1.0  0.0  1.0  0.0
Genotype4  0.0  0.0  0.0  1.0  0.0  0.0  1.0
Genotype5  0.0  0.0  0.0  1.0  1.0  1.0  0.0
Genotype6  0.0  0.0  0.0  0.0  1.0  1.0  0.0
Genotype7  0.0  0.0  0.0  0.0  1.0  0.0  1.0
Genotype8  0

# Conversions to NetworkX and igraph

In [79]:
# Convert to an igraph object
G = ont.to_igraph()
print(G)

IGRAPH DN-- 15 16 --
+ attr: NodeType (v), name (v), EdgeType (e)
+ edges (vertex names):
A->S3, B->S3, C->S3, C->S4, D->S4, E->S5, F->S5, G->S6, H->S6, S1->S0, S2->S0,
S3->S1, S4->S1, S5->S1, S5->S2, S6->S2


In [92]:
# Convert to an igraph object
G = ont.to_igraph(spanning_tree=True)
print(G)

IGRAPH DN-- 15 16 --
+ attr: NodeType (v), name (v), EdgeType (e), Is_Tree_Edge (e)
+ edges (vertex names):
A->S3, B->S3, C->S3, C->S4, D->S4, E->S5, F->S5, G->S6, H->S6, S1->S0, S2->S0,
S3->S1, S4->S1, S5->S1, S5->S2, S6->S2


In [93]:
# Reconstruct the Ontology object from the igraph object
Ontology.from_igraph(G)

8 genes, 7 terms, 9 gene-term relations, 7 term-term relations
node_attributes: []
edge_attributes: []

In [94]:
# Convert to a NetworkX object
G = ont.to_networkx()
print(G.nodes())
print(G.edges())

['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'S0', 'S1', 'S2', 'S3', 'S4', 'S5', 'S6']
[('A', 'S3'), ('B', 'S3'), ('C', 'S3'), ('C', 'S4'), ('D', 'S4'), ('E', 'S5'), ('F', 'S5'), ('G', 'S6'), ('H', 'S6'), ('S1', 'S0'), ('S2', 'S0'), ('S3', 'S1'), ('S4', 'S1'), ('S5', 'S1'), ('S5', 'S2'), ('S6', 'S2')]


In [95]:
# Reconstruct the Ontology object from the NetworkX object
tmp = Ontology.from_networkx(G)
print(tmp)

8 genes, 7 terms, 9 gene-term relations, 7 term-term relations
node_attributes: []
edge_attributes: []


# Ontology visualization using HiView (http://hiview.ucsd.edu)
* HiView is a web application for general visualization of the hierarchical structure in ontologies.
* First upload your ontology into NDEx using the [Ontology.to_ndex()](http://ddot.readthedocs.io/en/latest/ontology.html#ddot.Ontology.to_ndex) function, and then have HiView read the ontology from NDEx
* It supports ontologies that have a simple tree structure and those that are more complicated (directed acyclic graphs, in which nodes have multiple parents)

### A simple upload to NDEx and visualization in HiView
* Upload ontologies to NDEx using the `Ontology.to_ndex()` function.
* Setting the parameter `layout="bubble"` will identify a spanning tree of the DAG and then lay this tree in a space-compact manner. When viewing in HiView, only the edges in the spanning tree are shown, while the other edges can be chosen to be shown.

<img src="https://raw.githubusercontent.com/michaelkyu/ddot/master/docs/toy_ontology_spanning_tree.png" height="250" align="left">

In [96]:
url, _ = ont.to_ndex(ndex_server=ndex_server,
                     ndex_user=ndex_user,
                     ndex_pass=ndex_pass,
                     layout='bubble')

print('Go to http://hiview.ucsd.edu in your web browser')
print('Enter this into the "NDEx Sever URL" field: %s' % ndex_server)
print('Enter this into the "UUID of the main hierarchy" field: %s' % url.split('/')[-1])

Go to http://hiview.ucsd.edu in your web browser
Enter this into the "NDEx Sever URL" field: http://test.ndexbio.org
Enter this into the "UUID of the main hierarchy" field: e3d8cbb3-5fe7-11e8-9d1c-0660b7976219


### An alternative layout by duplicating nodes
* Setting the parameter `layout="bubble-collect"` will convert the DAG into a tree by duplicating nodes.
* This transformation enables the ontology structure to be visualized without edges crossing.

<img src="https://raw.githubusercontent.com/michaelkyu/ddot/master/docs/toy_ontology_dag2tree.png" height="250" align="left">

In [97]:
url, _ = ont.to_ndex(ndex_server=ndex_server,
                     ndex_user=ndex_user,
                     ndex_pass=ndex_pass,
                     layout='bubble-collect')

print('Go to http://hiview.ucsd.edu in your web browser')
print('Enter this into the "NDEx Sever URL" field: %s' % ndex_server)
print('Enter this into the "UUID of the main hierarchy" field: %s' % url.split('/')[-1])

Go to http://hiview.ucsd.edu in your web browser
Enter this into the "NDEx Sever URL" field: http://test.ndexbio.org
Enter this into the "UUID of the main hierarchy" field: e6b6be56-5fe7-11e8-9d1c-0660b7976219


### Visualizing metadata by modifying node labels, colors, and sizes

* An Ontology object has a `node_attr` field that is a pandas DataFrame. The rows of the dataframe are genes or terms, and the columns are node attributes.
* HiView understands special node attributes to control the node labels, colors, and sizes.

In [98]:
# Set the node labels (default is the gene and term names, as found in Ontology.genes and Ontology.terms)
ont.node_attr.loc['S4', 'Label'] = 'S4 alias'
ont.node_attr.loc['S5', 'Label'] = 'S5 alias'

# Set the fill color of nodes
ont.node_attr.loc['C', 'Vis:Fill Color'] = '#7fc97f'
ont.node_attr.loc['S1', 'Vis:Fill Color'] = '#beaed4'
ont.node_attr.loc['S0', 'Vis:Fill Color'] = '#fdc086'

# Set the node sizes (if not set, the default is the term size, as found in Ontology.term_sizes)
ont.node_attr.loc['C', 'Size'] = 10

ont.node_attr

,Label,Vis:Fill Color,Size
Node,,,
S4,S4 alias,NaN,NaN
S5,S5 alias,NaN,NaN
C,NaN,#7fc97f,10.0
S1,NaN,#beaed4,NaN
S0,NaN,#fdc086,NaN


In [99]:
url, _ = ont.to_ndex(ndex_server=ndex_server,
                     ndex_user=ndex_user,
                     ndex_pass=ndex_pass,
                     layout='bubble-collect')

print('Go to http://hiview.ucsd.edu in your web browser')
print('Enter this into the "NDEx Sever URL" field: %s' % ndex_server)
print('Enter this into the "UUID of the main hierarchy" field: %s' % url.split('/')[-1])

Go to http://hiview.ucsd.edu in your web browser
Enter this into the "NDEx Sever URL" field: http://test.ndexbio.org
Enter this into the "UUID of the main hierarchy" field: fe2a0249-5fe7-11e8-9d1c-0660b7976219


In [100]:
# Clear node attributes (optional)
ont.clear_node_attr()
ont.node_attr

""
Node


### Visualize gene-gene interaction networks alongside the ontology
* Every term in an ontology represents a biological function shared among the term's genes. Based on this intuition, those genes should be interacting in different ways, e.g. protein-protein interactions, RNA expression, or genetic interactions.
* Gene-gene interaction networks can be uploaded with the ontology to NDEx, so that they can be visualized at the same time in HiView

In [101]:
# Calculate a gene-by-gene similarity matrix using the Resnik semantic similarity definition (see section "Inferring a data-driven ontology")
sim, genes = ont.flatten()
print(genes)
print(sim)

['A' 'B' 'C' 'D' 'E' 'F' 'G' 'H']
[[ 1.4150375  1.4150375  1.4150375  0.4150375  0.4150375  0.4150375 -0.        -0.       ]
 [ 1.4150375  1.4150375  1.4150375  0.4150375  0.4150375  0.4150375 -0.        -0.       ]
 [ 1.4150375  1.4150375  2.         2.         0.4150375  0.4150375 -0.        -0.       ]
 [ 0.4150375  0.4150375  2.         2.         0.4150375  0.4150375 -0.        -0.       ]
 [ 0.4150375  0.4150375  0.4150375  0.4150375  2.         2.         1.         1.       ]
 [ 0.4150375  0.4150375  0.4150375  0.4150375  2.         2.         1.         1.       ]
 [-0.        -0.        -0.        -0.         1.         1.         2.         2.       ]
 [-0.        -0.        -0.        -0.         1.         1.         2.         2.       ]]


In [29]:
# Convert the gene-by-gene similarity matrix into a dataframe with a "long" format, where rows represent gene pairss
import pandas as pd
sim_df = pd.DataFrame(sim, index=genes, columns=genes)
sim_long = ddot.melt_square(sim_df)
sim_long.head()

,Gene1,Gene2,similarity
0,A,B,1.415038
1,A,C,1.415038
2,A,D,0.415038
3,A,E,0.415038
4,A,F,0.415038


In [27]:
# Create other gene-gene interactions. For example, these can represent protein-protein interactions or gene co-expression. Here, we simulate interactions by adding a random noise to the Resnik similarity
sim_long['Interaction_Type1'] = sim_long['similarity'] + np.random.random(sim_long.shape[0]) / 2.
sim_long['Interaction_Type2'] = sim_long['similarity'] + np.random.random(sim_long.shape[0]) / 2.
sim_long.head()

,Gene1,Gene2,similarity,Interaction_Type1,Interaction_Type2
0,A,B,1.415038,1.833367,1.618822
1,A,C,1.415038,1.668998,1.727922
2,A,D,0.415038,0.766172,0.643903
3,A,E,0.415038,0.664731,0.585459
4,A,F,0.415038,0.669328,0.571242


In [37]:
# Include the above gene-gene interactions by setting the `network` and `main_feature` parameters.
url, _ = ont.to_ndex(ndex_server=ndex_server,
                     ndex_user=ndex_user,
                     ndex_pass=ndex_pass,
                     network=sim_long,
                     main_feature='similarity',
                     layout='bubble-collect')

print('Go to http://hiview.ucsd.edu in your web browser')
print('Enter this into the "NDEx Sever URL" field: %s' % ndex_server)
print('Enter this into the "UUID of the main hierarchy" field: %s' % url.split('/')[-1])

Go to http://hiview.ucsd.edu in your web browser
Enter this into the "NDEx Sever URL" field: http://test.ndexbio.org
Enter this into the "UUID of the main hierarchy" field: a0770167-5fab-11e8-9d1c-0660b7976219
